In [4]:
#dependency
import os 
import json
import petpy
import urllib
import pandas as pd

import time
import giphy_client
from giphy_client.rest import ApiException
from pprint import pprint

#database 
from sqlalchemy import create_engine

#
from config import key, secret, api_key 
import config



In [5]:
# initialized
pf = petpy.Petfinder(key=key, secret=secret)

In [ ]:
pf

In [ ]:
#find breed type 
#German Shepherd and ragdoll cats.
dog_breeds = pf.breeds('dog')
dog_breeds

In [ ]:
cat = pf.breeds('cat')
cat

In [7]:
# local adoption information cats.
cat_df = pf.animals(animal_type='cat', status='adoptable', location='Miami, FL', distance=10,
                  results_per_page=5, pages=2, return_df=True)
cat_df

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,animal_id,animal_type,organization_id
0,52692465,FL1694,https://www.petfinder.com/cat/nikki-52692465/f...,Cat,Cat,Baby,Female,Small,Short,"[Friendly, Affectionate, Playful, Independent,...",...,None,None,None,Miami Beach,FL,33139,US,52692465,cat,fl1694
1,52660749,FL1729,https://www.petfinder.com/cat/tabica-52660749/...,Cat,Cat,Baby,Female,Medium,Short,"[Friendly, Affectionate, Gentle, Playful]",...,(305) 303-6335,None,None,Miami,FL,33155,US,52660749,cat,fl1729
2,52660598,FL1729,https://www.petfinder.com/cat/jellybean-526605...,Cat,Cat,Baby,Male,Medium,Short,"[Friendly, Affectionate, Gentle, Playful]",...,(305) 303-6335,None,None,Miami,FL,33155,US,52660598,cat,fl1729
3,52654087,FL1729,https://www.petfinder.com/cat/olivia-52654087/...,Cat,Cat,Baby,Female,Medium,Short,"[Friendly, Affectionate, Playful, Loves kisses...",...,(305) 303-6335,None,None,Miami,FL,33155,US,52654087,cat,fl1729
4,52654050,FL1729,https://www.petfinder.com/cat/charlie-52654050...,Cat,Cat,Baby,Male,Medium,Short,"[Friendly, Affectionate, Playful, Loves kisses...",...,(305) 303-6335,None,None,Miami,FL,33155,US,52654050,cat,fl1729
5,52654026,FL1729,https://www.petfinder.com/cat/teddy-52654026/f...,Cat,Cat,Baby,Male,Medium,Short,"[Friendly, Affectionate, Gentle, Playful, Love...",...,(305) 303-6335,None,None,Miami,FL,33155,US,52654026,cat,fl1729
6,52543838,FL62,https://www.petfinder.com/cat/xenna-52543838/f...,Cat,Cat,Baby,Female,Medium,Short,"[Playful, Smart, Loves kisses, Friendly, Affec...",...,(305) 255-3482,None,None,Miami,FL,33234,US,52543838,cat,fl62
7,52543815,FL62,https://www.petfinder.com/cat/cautious-5254381...,Cat,Cat,Young,Female,Medium,Short,[],...,(305) 255-3482,None,None,Miami,FL,33234,US,52543815,cat,fl62
8,52536504,FL62,https://www.petfinder.com/cat/charlie-52536504...,Cat,Cat,Young,Male,Medium,Short,[],...,(305) 255-3482,None,None,Miami,FL,33234,US,52536504,cat,fl62
9,52456261,FL1729,https://www.petfinder.com/cat/emma-52456261/fl...,Cat,Cat,Baby,Female,Small,Short,"[Friendly, Affectionate, Loyal, Gentle, Playfu...",...,(305) 303-6335,None,None,Miami,FL,33155,US,52456261,cat,fl1729


In [14]:
#Filtered the data 
new_cat_df = cat_df[['id','organization_id','species','age','gender','animal_id','contact.address.city','contact.phone'] ]

#set index to id
# new_cat_df = new_cat_df.set_index('id')
# new_cat_df

In [15]:
#rename columns
new_cat_df = new_cat_df.rename(columns={'contact.address.city':'city',
                                        'contact.phone':'phone_number'})
new_cat_df

,id,organization_id,organization_id,species,age,gender,animal_id,city,phone_number
0,52692465,FL1694,fl1694,Cat,Baby,Female,52692465,Miami Beach,None
1,52660749,FL1729,fl1729,Cat,Baby,Female,52660749,Miami,(305) 303-6335
2,52660598,FL1729,fl1729,Cat,Baby,Male,52660598,Miami,(305) 303-6335
3,52654087,FL1729,fl1729,Cat,Baby,Female,52654087,Miami,(305) 303-6335
4,52654050,FL1729,fl1729,Cat,Baby,Male,52654050,Miami,(305) 303-6335
5,52654026,FL1729,fl1729,Cat,Baby,Male,52654026,Miami,(305) 303-6335
6,52543838,FL62,fl62,Cat,Baby,Female,52543838,Miami,(305) 255-3482
7,52543815,FL62,fl62,Cat,Young,Female,52543815,Miami,(305) 255-3482
8,52536504,FL62,fl62,Cat,Young,Male,52536504,Miami,(305) 255-3482
9,52456261,FL1729,fl1729,Cat,Baby,Female,52456261,Miami,(305) 303-6335


In [32]:
new_cat_df_x = []
id_set = set()

dups = set()

for item in new_cat_df.iterrows():
    print(item[1].id)
    
    if item[1].id not in id_set:
        new_cat_df_x.append(item)
    else:
        dups.add(item[1].id)
        
    id_set.add(item[1].id)

print(dups)
    

52692465
52660749
52660598
52654087
52654050
52654026
52543838
52543815
52536504
52456261
set()


In [20]:
new_cat_df['id'][0]

52692465

# Load the data into SQL Database


In [33]:
# Create a SQL Database connection
connection_string = f"postgres:{config.password}@localhost:5432/pets_db"
engine = create_engine(f'postgresql://{connection_string}')

In [34]:
# checking tables
engine.table_names()

['stage_dog_db', 'dog_db', 'cat2_db', 'cat_db', 'stage_cat_db']

In [ ]:
try:
    new_cat_df.to_sql(name="cat_db", con=engine, if_exists="append", index=True)
    print("Data loaded successfully")
    
except:
    print("Data has already been loaded to db")

In [ ]:

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
import requests
import matplotlib.pyplot as plt
from config import key, secret, api_key
import pandas as pd
import json
import time

In [ ]:
#create base url & query url
base_url = 'https://api.petfinder.com/v2/'

animal = 'animals'
animal_type = 'dog'
page = '2'

query_url = base_url + animal +'?type=' + animal_type +'&page='+ page
print(query_url)

In [ ]:
curl -H "Authorization: Bearer eyJ0eXA..." https://api.petfinder.com/v2/animals?type=dog&page=2

Store day's data in database 

Giphy API


In [ ]:
url = 'api.giphy.com/v1/gifs/trending'


In [ ]:
params = urllib.parse.urlencode({
    'q': 'cats',
    'api_key':api_key,
    'limit':'10',
    'url': url  
})

with urllib.request.urlopen("".join(params)) as response:
    data = json.loads(response.read())
    
print(json.dumps(data, sort_keys=True, indent=4))

In [ ]:

import time
import giphy_client
from giphy_client.rest import ApiException
from pprint import pprint

# create an instance of the API class
api_instance = giphy_client.DefaultApi()
api_key = 'oXxZbqThXjFhCUEXrrQOeJqmHTAxyjSn'
limit = 25 # int | The maximum number of records to return. (optional) (default to 25)
rating = 'g' # str | Filters results by specified rating. (optional)
fmt = 'json' 

try: 
    # Trending GIFs Endpoint
    api_response = api_instance.gifs_trending_get(api_key, limit=limit, rating=rating, fmt=fmt)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling DefaultApi->gifs_trending_get: %s\n" % e)